In [1]:
import numpy as np
import sys
import copy
import ktms

# env = 'local'
env = 'spacom'
surface = 'Cu'
adsorbate = 'CO'

mirror = '100'
unitlength = 1
molenum = 1 # number of molecules
# mindistlim = 2.5

##################################################################################
### Set coefficients ###
d = 1.13
h = 1.85
layers = 4
# fixlayer = 3
latticeconstant = 4.21027100913
# Cu = 3.67676461295 # previously 3.67674038800827
# Pt = 3.99042863796
# Ag = 4.21027100913

unique = ''

if molenum == 1:
    symred = True
else:
    symred = False

if mirror =='100':
    initatoms = ktms.fcc100(surface, a=latticeconstant, size=(unitlength, unitlength, layers), vacuum=10.0)
elif mirror == '111':
    initatoms = ktms.fcc111(surface, a=latticeconstant, size=(unitlength, unitlength, layers), vacuum=10.0)

### Set molecule ###
name = adsorbate + unique +'.traj'

if ktms.init_query(name, env) != 'No file':
    molecule = ktms.init_query(name, env)
    print(name, 'read')
else:
    molecule = ktms.Atoms(adsorbate, positions=[(0., 0., 0.), (0., 0., d)], cell=(10., 10., 10.))
    trajpath = '/home/katsuyut/init/' + str(name)
    molecule.write(trajpath)
    print(name, 'created!')


### Set slab ###
name = surface + mirror + '_unit' + str(unitlength) + unique + '.traj'

if ktms.init_query(name, env) != 'No file':
    atoms = ktms.init_query(name, env)
    print(name, 'read')
else:
    atoms = initatoms
    trajpath = '/home/katsuyut/init/' + str(name)
#     atoms.write(trajpath)
    print(name, 'created!')

cell = atoms.cell

### Set adsorption sites ###
sites = ktms.getadsites(atoms, symred) # all sites
positions = ktms.getadsitecomb(sites, molenum) # combination of sites with n molecules
calccombs, uniquecomb, results = ktms.getcalccomb(sites, molenum, cell) # reduced combinations


### Set adsorbed slab ###
for i in range(len(calccombs)):
    for j in range(len(calccombs[i])):
        if type(calccombs[i][j]) != int:
            
            sitetype = ktms.checksitetype(calccombs[i][j], sites)
            name = surface + mirror + '_unit' + str(unitlength) + '_' + adsorbate + '_' + str(sitetype[0]) + str(sitetype[1]) \
                + str(sitetype[2]) + '_' + str(int(ktms.np.ceil(ktms.getmindist(calccombs[i][j], atoms.cell)/0.5)-3)) \
                + unique + '.traj'
            trajpath = '/home/katsuyut/init/' + str(name)

            if ktms.init_query(name, env) != 'No file':
                atoms = ktms.init_query(name, env)
                print(name, 'read')
            else:
                watoms = copy.deepcopy(atoms)
                ktms.getcalcatoms(watoms, molecule, h, calccombs[i][j])
#                 watoms.write(trajpath)
                print(name, 'created!')

CO.traj created!
Cu100_unit1.traj created!
No hollow
No hollow
Cu100_unit1_CO_100_3.traj created!
Cu100_unit1_CO_010_3.traj created!
